In [ ]:
import numpy as np
import telegram_send
import cv2,time
from datetime  import datetime
import winsound


# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

# open webcam video stream
cap = cv2.VideoCapture(0)


##variables
detection = False
detection_stopped_time = None
timer_started = False
SECONDS_TO_RECORD_AFTER_DETECTION = 5
count=0

while(True):
   
    # Capture frame-by-frame
    ret, frame = cap.read()

    # resizing for faster detection
    frame = cv2.resize(frame, (480,640))
    # using a greyscale picture, also for faster detection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # detect people in the image
    # returns the bounding boxes for the detected objects
    boxes,  weights =  hog.detectMultiScale(frame, winStride = (4, 4), padding = (8, 8), scale = 1.03)
    


    ## recording
    
    if len(weights)  > 0:
        winsound.Beep(500,200)

        if detection:
            timer_started = False
        else:
            detection = True
            current_time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        

            out = cv2.VideoWriter(f"{current_time}.avi",cv2.VideoWriter_fourcc(*'MJPG'),15.,(480,640))    
            print("Started Recording!")
            count=1

    elif detection:
        if timer_started:
            if time.time() - detection_stopped_time >= SECONDS_TO_RECORD_AFTER_DETECTION:
                detection = False
                timer_started = False
                out.release()
                print('Stop Recording!')
        else:
            timer_started = True
            detection_stopped_time = time.time()

    if detection:
        out.write(frame)
    ##Alert system
    if count == 1:
        telegram_send.send(messages=["detectedsdsds"])
        #current_time = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
        with open("face detected   "+str(exact_time)+".jpg", "rb") as f:
            telegram_send.send(images=[f])
        count = 0



     ## drawing box...............................................................

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    for (xA, yA, xB, yB) in boxes:
        # display the detected boxes in the colour picture
        img=cv2.rectangle(frame, (xA, yA), (xB, yB),(0, 255, 0), 2)
        cv2.imwrite('detected'+str(0)+'.png', img)
        exact_time=datetime.now().strftime('%Y-%b-%d-%H-%S')
        cv2.imwrite("face detected   "+str(exact_time)+".jpg",img)
       
        #cv2.putText(frame, f'person {person}', (xA,yA), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)


    


        
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    #cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('SMART SURVILLANCE SYSTEM',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


        
# When everything done, release the capture
cap.release()
# and release the output
out.release()
# finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(1)